In [ ]:
# Define dataset path
dataset_path = "/kaggle/input/dataset/inaturalist_12K"

In [ ]:
import wandb
wandb.login(key="5fb34431b405eb21dc0f263e5b3cf2c15fdc7471")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import wandb
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
# Define dataset path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = "/kaggle/input/dataset/inaturalist_12K"

def get_dataloaders(dataset_path, augment, batch_size=32):
    # Define transforms
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ]) if augment else transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    full_dataset = ImageFolder(root=f"{dataset_path}/train", transform=transform)

    # ✅ Class-balanced 80/20 split
    
    targets = np.array(full_dataset.targets)
    strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_idx, val_idx in strat_split.split(np.zeros(len(targets)), targets):
        train_subset = torch.utils.data.Subset(full_dataset, train_idx)
        val_subset = torch.utils.data.Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader

train_loader, val_loader = get_dataloaders(dataset_path, augment=True, batch_size=32)


In [ ]:


import torch
import torch.nn as nn

class CNN_Model(nn.Module):
    def __init__(self, 
                 filter_sizes,              # list of conv filters
                 dense_neurons,             # neurons in dense layer
                 activation="relu",         # activation function
                 dropout=0.0,               # dropout rate
                 batch_norm=False,          # whether to use BatchNorm
                 input_shape=(3, 224, 224), # image shape
                 num_classes=10):           # number of output classes
        super(CNN_Model, self).__init__()

        # Activation function class (module, not functional)
        act_layer = {
            "relu": nn.ReLU,
            "gelu": nn.GELU,
            "silu": nn.SiLU,
            "mish": nn.Mish
        }[activation.lower()]
        
        in_channels = input_shape[0]
        layers = []

        # Convolution blocks
        for i, out_channels in enumerate(filter_sizes):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))  # padding=1 to preserve size
            if batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(act_layer())
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))  # downsample by 2
            if dropout > 0:
                layers.append(nn.Dropout2d(dropout))
            in_channels = out_channels

        self.conv_blocks = nn.Sequential(*layers)

        # Automatically infer flatten size
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            flatten_dim = self.conv_blocks(dummy).view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Linear(flatten_dim, dense_neurons),
            act_layer(),
            nn.Dropout(dropout) if dropout > 0 else nn.Identity(),
            nn.Linear(dense_neurons, num_classes)
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x


In [ ]:
# ✅ Initialize WandB (resumes existing run if needed)
wandb.init(project="Assignment_2_CNN", resume=True)
def train_cnn_model(model, train_loader, val_loader, config, device):
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    best_val_acc = 0.0

    for epoch in range(config["epochs"]):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        val_loss /= len(val_loader)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")

        # Logging to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })

        print(f"Epoch {epoch+1} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    wandb.run.summary["best_val_accuracy"] = best_val_acc
    return model
wandb.finish()

In [ ]:

sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": [10,15,20]},
        "filter_sizes": {
            "values": [
                [16, 32, 32, 32, 256],
                [128, 128, 128, 128, 128],
                [32, 64, 64, 128, 128],
                [32, 64, 128, 128, 256],
            ]
        },
        "dense_neurons": {"values": [128, 256, 512]},
        "activation": {"values": ["gelu","relu", "silu", "mish"]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["adam"]},
        "dropout": {"values": [0.2, 0.3]},
        "data_augmentation": {"values": [True, False]},
        "batch_norm": {"values": [True, False]},
        "weight_decay": {"values": [0, 0.0005]},
    }
}


In [ ]:

def train_wandb():
    with wandb.init():
        config = wandb.config

        # ✅ Load dataset
        dataset_path = "/kaggle/input/dataset/inaturalist_12K"
        train_loader, val_loader = get_dataloaders(
            dataset_path=dataset_path,
            augment=config.data_augmentation,
            batch_size=getattr(config, "batch_size", 32)
        )

        # ✅ Build model from config
        model = CNN_Model(
            filter_sizes=config.filter_sizes,
     
            
            dense_neurons=config.dense_neurons,
            activation=config.activation,
            dropout=config.dropout,
            batch_norm=config.batch_norm,
            input_shape=(3, 224, 224),
            num_classes=10  
        ).to(device)

        # ✅ Optional: log model internals
        wandb.watch(model, log="all", log_freq=100)

        # ✅ Train with early stopping
        train_cnn_model(model, train_loader, val_loader, config, device)

# Launch sweep
sweep_id = wandb.sweep(sweep_config, project="Assignment_2_CNN")
wandb.agent(sweep_id, function=train_wandb, count=15)


# Testing With Saved model

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import wandb  # Add this

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = "/kaggle/input/dataset/inaturalist_12K"

# Transformation: Resize, Convert to Tensor, and Normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the test dataset
test_dataset = ImageFolder(root=f"{dataset_path}/val", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

# Get class names
class_names = test_dataset.classes
print(f"Test samples: {len(test_dataset)}")

# Initialize the model
model = CNN_Model(
    filter_sizes=[128, 128, 128, 128, 128],  
    dense_neurons=256,  
    activation="mish", 
    dropout=0.2, 
    batch_norm=True,  
    input_shape=(3, 224, 224),
    num_classes=10  
).to(device)

# Load the best model weights
model.load_state_dict(torch.load("best_model.pth", weights_only=True))
model.eval()

# Evaluation on test data
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

wandb.init(project="Assignment_2_CNN", resume=True)
# ✅ Log accuracy to wandb
wandb.log({"Test Accuracy": test_accuracy})




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import wandb
import torch

# Collect 30 test samples and their predictions
model.eval()
images_shown = 0
images_to_show = 30
images_list, preds_list, labels_list = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        for i in range(images.size(0)):
            if images_shown >= images_to_show:
                break
            images_list.append(images[i].cpu())
            preds_list.append(predicted[i].cpu())
            labels_list.append(labels[i].cpu())
            images_shown += 1

        if images_shown >= images_to_show:
            break

# Denormalize images
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
images_list = [(img.permute(1, 2, 0) * std + mean).numpy() for img in images_list]

# Plot 10×3 grid with color-coded titles
fig, axes = plt.subplots(10, 3, figsize=(10, 30))
for i, ax in enumerate(axes.flat):
    ax.imshow(np.clip(images_list[i], 0, 1))
    true_label = class_names[labels_list[i]]
    pred_label = class_names[preds_list[i]]

    title_color = "black" if true_label == pred_label else "red"
    ax.set_title(f"True: {true_label}\nPred: {pred_label}", fontsize=8, color=title_color)
    ax.axis("off")

plt.tight_layout()

# Log to wandb
# wandb.log({"Test Predictions Grid": wandb.Image(fig)})
# Save the figure first
fig.savefig("test_predictions_grid.png")

# Then log it from file
wandb.log({"Test Predictions Grid": wandb.Image("test_predictions_grid.png")})



# Show the figure in notebook
plt.show()
plt.close(fig)
